<a href="https://colab.research.google.com/github/ThaumielSparrow/LSD/blob/main/LSD_training/Rotation_Sarah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import numpy as np
import imutils
from matplotlib import pyplot as plt
from math import atan2, cos, sin, sqrt, pi

In [ ]:
class CenterAndRotationWithPCA():
  '''
  Contains structure to make digit detection scalable for the future

  Functions: digits_by_contour
  '''
  def __init__(self, img):
    self.image = img

  def drawAxis(self, img, p_, q_, color, scale):
    p = list(p_)
    q = list(q_)
 
    ## [visualization1]
    angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
    hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))
 
    # Here we lengthen the arrow by a factor of scale
    q[0] = p[0] - scale * hypotenuse * cos(angle)
    q[1] = p[1] - scale * hypotenuse * sin(angle)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)
 
    # create the arrow hooks
    p[0] = q[0] + 9 * cos(angle + pi / 4)
    p[1] = q[1] + 9 * sin(angle + pi / 4)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)
 
    p[0] = q[0] + 9 * cos(angle - pi / 4)
    p[1] = q[1] + 9 * sin(angle - pi / 4)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)
    ## [visualization1]
 
  def preprocess_image(self) :
    '''
    Read the image, reduce noise, threshhold and dilate
    pre: self.image is just a file name
    post: self.image is preprocessed
    '''
    img_main = cv.imread(self.image)
    img_blur = cv.GaussianBlur(img_main, (7, 7), 1)
    img_blur_RGB = cv.cvtColor(img_blur, cv.COLOR_BGR2GRAY)

    ret, thresh1 = cv.threshold(img_blur_RGB, 127, 256, cv.THRESH_BINARY_INV)
    dilate = cv.dilate(thresh1, None, iterations=2)
    return dilate
    
  def getOrientation(self, pts, img):
    ## [pca]
    # Construct a buffer used by the pca analysis
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
      data_pts[i,0] = pts[i,0,0]
      data_pts[i,1] = pts[i,0,1]
 
    # Perform PCA analysis
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv.PCACompute2(data_pts, mean)
 
    # Store the center of the object
    cntr = (int(mean[0,0]), int(mean[0,1]))
    ## [pca]
 
    ## [visualization]
    # Draw the principal components
    cv.circle(img, cntr, 3, (255, 0, 255), 2)
    p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
    p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
    self.drawAxis(img, cntr, p1, (255, 255, 0), 1)
    self.drawAxis(img, cntr, p2, (0, 0, 255), 5)
 
    angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
    ## [visualization]
 
    # Label with the rotation angle
    label = "  Rotation Angle: " + str(-int(np.rad2deg(angle)) - 90) + " degrees"
    textbox = cv.rectangle(img, (cntr[0], cntr[1]-25), (cntr[0] + 250, cntr[1] + 10), (255,255,255), -1)
    cv.putText(img, label, (cntr[0], cntr[1]), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv.LINE_AA)
 
    return cntr, angle

  def image_moments(self):
    '''
    Finds digits in an image by contours.

    Arguments: image to be analyzed, whether to save images physically, whether to return original image with bounds around digits

    Returns: NumPy array containing all digits
    '''

    img = cv.imread(self.image)
    contours = []
    dilate = self.preprocess_image();
    cntrs, _ = cv.findContours(dilate, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
    for i, c in enumerate(cntrs):
      # Calculate the area of each contour
      area = cv.contourArea(c)
      if area < 100:
        continue
      # Draw each contour only for visualisation purposes
      cv.drawContours(img, contours, i, (0, 0, 255), 2)
 
    # Find the orientation of each shape
    cntr, angle = self.getOrientation(c, img)
    #cv.imshow('Output Image', img)
    #cv.waitKey(0)
    #cv.destroyAllWindows()
    cv.imwrite("/output_img.png", img)
    
        
    

In [ ]:
rot = CenterAndRotationWithPCA("/image.png");
rot.image_moments()